# Pytorch Module

pytorch module 是一個 pytorch 寫好的物件，當我們想要自定義網路的時候必須從 ```nn.Module``` 繼承 pytorch module 的屬性，在自定義網路中必須包含兩個部分

- ```__init__():``` 網路初始化的設定
- ```forward():``` 前向傳播的方法，回傳模型的輸出

另外 nn 底下的 built-in 函數都會自動放在 ```nn.Parameter``` 裡面，所以不用特定再聲明

我們先來看一個非常簡單的例子

In [ ]:
import torch.nn as nn
import torch

from torchsummary import summary

class MyModule(nn.Module):

    def __init__(self, in_features=128, out_features=64):
        super().__init__()

        self.weight = nn.Linear(in_features, out_features)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.weight(x)
        x = self.activation(x)
        return x

if __name__ == '__main__':
    x = torch.randn(3,128)
    module = MyModule()
    print('output size:', module(x).shape)
    summary(module, (3, 128))
    print(module)

output size: torch.Size([3, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 3, 64]           8,256
           Sigmoid-2                [-1, 3, 64]               0
Total params: 8,256
Trainable params: 8,256
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.04
----------------------------------------------------------------
MyModule(
  (weight): Linear(in_features=128, out_features=64, bias=True)
  (activation): Sigmoid()
)


在這個例子中我們的網路有兩層

- weight: 會將 input data 乘上一個大小為 128 * 64 的矩陣，再加上維度為 64 的 bias 向量
- activation: 將 Linear 層的 output 通過一個 sigmoid 函數，保證 output 的值落在 [0, 1] 區間內

全部參數量 8256 是怎麼計算出來的呢?

- 網路的計算是 $\sigma(WX+B)$，其中 $W、B$ 是可更新參數
- 所以全部參數量為 64 * 128 + 64


## Sequential

如果我們想要更多層的神經網路呢? 我們可以使用 Sequential 方法，Sequential 會將一連串的函數傳接再一起，並且自動生成計算圖的路徑 (自動生成 forward 路徑)

In [ ]:
class MyModule(nn.Module):

    def __init__(self, in_features=128, out_features=64):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(in_features, in_features // 2),
            nn.BatchNorm1d(in_features // 2),
            nn.ReLU(),
            nn.Linear(in_features // 2, out_features),
            nn.BatchNorm1d(out_features),
        )
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.layers(x)
        x = self.activation(x)
        return x

if __name__ == '__main__':
    x = torch.randn(64,128)
    module = MyModule()
    print('output size:', module(x).shape)
    summary(module, (64, 128))
    print(module)

output size: torch.Size([64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 64, 64]           8,256
       BatchNorm1d-2               [-1, 64, 64]             128
              ReLU-3               [-1, 64, 64]               0
            Linear-4               [-1, 64, 64]           4,160
       BatchNorm1d-5               [-1, 64, 64]             128
           Sigmoid-6               [-1, 64, 64]               0
Total params: 12,672
Trainable params: 12,672
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.19
Params size (MB): 0.05
Estimated Total Size (MB): 0.27
----------------------------------------------------------------
MyModule(
  (layers): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentu

除了直接寫在 Sequential 裡面，我們也可以寫在 List 裡面再展開

In [ ]:
class MyModule(nn.Module):

    def __init__(self, in_features=128, out_features=64):
        super().__init__()

        layers = [
            nn.Linear(in_features, in_features // 2),
            nn.BatchNorm1d(in_features // 2),
            nn.ReLU(),
            nn.Linear(in_features // 2, out_features),
            nn.BatchNorm1d(out_features),
        ]

        self.layers = nn.Sequential(*layers)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.layers(x)
        x = self.activation(x)
        return x

if __name__ == '__main__':
    x = torch.randn(64,128)
    module = MyModule()
    print('output size:', module(x).shape)
    summary(module, (64, 128))
    print(module)

output size: torch.Size([64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 64, 64]           8,256
       BatchNorm1d-2               [-1, 64, 64]             128
              ReLU-3               [-1, 64, 64]               0
            Linear-4               [-1, 64, 64]           4,160
       BatchNorm1d-5               [-1, 64, 64]             128
           Sigmoid-6               [-1, 64, 64]               0
Total params: 12,672
Trainable params: 12,672
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.19
Params size (MB): 0.05
Estimated Total Size (MB): 0.27
----------------------------------------------------------------
MyModule(
  (layers): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentu

### add_module

除了一個一個輸入以外，我們也可以使用 add_module 方法將參數註冊到網路中

add_module 方法需要輸入兩個參數，子類名稱和子類，由於 add_module 是 Sequential 的屬性，所以我們需要先建立 Sequential 物件才能使用 add_module 方法





In [ ]:
class MyModule(nn.Module):

    def __init__(self, in_features=512, out_features=64, depth=5):
        super().__init__()

        self.layers = nn.Sequential()
        for i in range(depth):
            self.layers.add_module(f'linear{i+1}', nn.Linear(in_features, in_features // 2))
            self.layers.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
            in_features = in_features // 2
        self.layers.add_module(f'linear{depth+1}', nn.Linear(in_features, out_features))

        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.layers(x)
        x = self.activation(x)
        return x

if __name__ == '__main__':
    x = torch.randn(64,512)
    module = MyModule()
    print('output size:', module(x).shape)
    summary(module, (64, 512))
    print(module)

output size: torch.Size([64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 64, 256]         131,328
              ReLU-2              [-1, 64, 256]               0
            Linear-3              [-1, 64, 128]          32,896
              ReLU-4              [-1, 64, 128]               0
            Linear-5               [-1, 64, 64]           8,256
              ReLU-6               [-1, 64, 64]               0
            Linear-7               [-1, 64, 32]           2,080
              ReLU-8               [-1, 64, 32]               0
            Linear-9               [-1, 64, 16]             528
             ReLU-10               [-1, 64, 16]               0
           Linear-11               [-1, 64, 64]           1,088
          Sigmoid-12               [-1, 64, 64]               0
Total params: 176,176
Trainable params: 176,176
Non-trainable params:

## Nested Module

通常一個常用的網路部件會被調用很多次，我們可以先把它寫成一個物件，以後每次要用到只需要調用物件就可

我們來看一個簡單的例子，假設我們的網路常常需要用到 ```Linear-BatchNorm-ReLU``` 的部件，那麼我就可以新定義一個物件 ```LBR``` 如下

In [ ]:
class LBR(nn.Module):

    def __init__(self, in_features, out_features):
        super().__init__()

        self.linear = nn.Linear(in_features, out_features)
        self.bn =  nn.BatchNorm1d(out_features)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.linear(x)
        x = self.bn(x)
        x = self.relu(x)

        return x

由於 nn 底下的函數都是 Module 的子類，所以我們可以向使用 ```nn.Linear``` 那樣直接使用 ```LBR```，不需要再用 ```nn.Parameter``` 告訴網路這是需要更新的參數

In [ ]:
class MyModule(nn.Module):

    def __init__(self, in_features=128, out_features=64):
        super().__init__()

        layers = [
            LBR(in_features, in_features // 2),
            LBR(in_features // 2, out_features),
        ]

        self.layers = nn.Sequential(*layers)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.layers(x)
        x = self.activation(x)
        return x

if __name__ == '__main__':
    x = torch.randn(64,128)
    module = MyModule()
    print('output size:', module(x).shape)
    summary(module, (64, 128))
    print(module)

output size: torch.Size([64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 64, 64]           8,256
       BatchNorm1d-2               [-1, 64, 64]             128
              ReLU-3               [-1, 64, 64]               0
               LBR-4               [-1, 64, 64]               0
            Linear-5               [-1, 64, 64]           4,160
       BatchNorm1d-6               [-1, 64, 64]             128
              ReLU-7               [-1, 64, 64]               0
               LBR-8               [-1, 64, 64]               0
           Sigmoid-9               [-1, 64, 64]               0
Total params: 12,672
Trainable params: 12,672
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.28
Params size (MB): 0.05
Estimated Total Size (MB): 0.36
-----------

## ModuleList

在 Sequential 方法中，Layer 之間是存在 **forward** 關係的，那如果我們只想蒐集某些部件，他們之間不存在連接關係呢? 這就可以用到 ModuleList 方法

ModuleList 方法會把 Module 的子類用 list 蒐集起來，並註冊到網路底下，但是子類彼此之間是獨立的

In [ ]:
class MyModule(nn.Module):

    def __init__(self, in_features=128, out_features=64):
        super().__init__()

        self.layers = nn.ModuleList([
            LBR(in_features, in_features // 2),
            LBR(in_features // 2, out_features),
        ])

        self.activation = nn.Sigmoid()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.activation(x)
        return x

if __name__ == '__main__':
    x = torch.randn(64,128)
    module = MyModule()
    print('output size:', module(x).shape)
    summary(module, (64, 128))
    print(module)

output size: torch.Size([64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 64, 64]           8,256
       BatchNorm1d-2               [-1, 64, 64]             128
              ReLU-3               [-1, 64, 64]               0
               LBR-4               [-1, 64, 64]               0
            Linear-5               [-1, 64, 64]           4,160
       BatchNorm1d-6               [-1, 64, 64]             128
              ReLU-7               [-1, 64, 64]               0
               LBR-8               [-1, 64, 64]               0
           Sigmoid-9               [-1, 64, 64]               0
Total params: 12,672
Trainable params: 12,672
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.28
Params size (MB): 0.05
Estimated Total Size (MB): 0.36
-----------

## Conclusion

在這一章節，我們學到了怎麼建立一個簡易的神經網路，除去一些特別的網路架構，本章節的內容可以處理大部分的網路問題了 !!

- 自定義網路須至少包含
  - ```__init__():``` 網路初始化設定
  - ```forward():``` 前向傳播路徑
- ```Sequential:``` 自動將子類註冊到網路中，且子類之間有 forward 關係
- ```ModuleList:``` 自動將子類註冊到網路中，子類之間相互獨立
- ```add_module:``` Sequential 的附屬方法
- 自定義子類，需繼承於 ```nn.Module```